In [2]:
import pandas as pd

In [4]:
products = pd.read_csv('/Users/paulbrenotvinciguerra/Desktop/amazon_baby.csv')

In [7]:
products = products.fillna({'review':''})  # fill in N/A's in the review column
def remove_punctuation(text):
    import string
    translator = str.maketrans('', '', string.punctuation)
    return text.translate(translator) 

products['review_clean'] = products['review'].apply(remove_punctuation)
products.head(5)

,name,review,rating,review_clean
0,Planetwise Flannel Wipes,"These flannel wipes are OK, but in my opinion ...",3,These flannel wipes are OK but in my opinion n...
1,Planetwise Wipe Pouch,it came early and was not disappointed. i love...,5,it came early and was not disappointed i love ...
2,Annas Dream Full Quilt with 2 Shams,Very soft and comfortable and warmer than it l...,5,Very soft and comfortable and warmer than it l...
3,Stop Pacifier Sucking without tears with Thumb...,This is a product well worth the purchase. I ...,5,This is a product well worth the purchase I h...
4,Stop Pacifier Sucking without tears with Thumb...,All of my kids have cried non-stop when I trie...,5,All of my kids have cried nonstop when I tried...


In [8]:
products = products[products['rating'] != 3]

In [9]:
products['sentiment'] = products['rating'].apply(lambda rating : +1 if rating > 3 else -1)
products.head(3)

,name,review,rating,review_clean,sentiment
1,Planetwise Wipe Pouch,it came early and was not disappointed. i love...,5,it came early and was not disappointed i love ...,1
2,Annas Dream Full Quilt with 2 Shams,Very soft and comfortable and warmer than it l...,5,Very soft and comfortable and warmer than it l...,1
3,Stop Pacifier Sucking without tears with Thumb...,This is a product well worth the purchase. I ...,5,This is a product well worth the purchase I h...,1


In [15]:
from sklearn.model_selection import train_test_split
train_data, test_data = train_test_split(products, test_size = 0.3, random_state=42)

In [16]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(token_pattern=r'\b\w+\b')
# Use this token pattern to keep single-letter words
# First, learn vocabulary from the training data and assign columns to words
# Then convert the training data into a sparse matrix
train_matrix = vectorizer.fit_transform(train_data['review_clean'])
# Second, convert the test data into a sparse matrix, using the same word-column mapping
test_matrix = vectorizer.transform(test_data['review_clean'])
#print vectorizer.vocabulary_

In [17]:
from sklearn.linear_model import LogisticRegression
sentiment_model = LogisticRegression()
sentiment_model.fit(train_matrix, train_data['sentiment'])

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [19]:
import numpy as np
np.sum(sentiment_model.coef_ >= 0)

78542

In [22]:
sample_test_data = test_data.iloc[10:13]
print(sample_test_data)

                                                    name  \
80054  Simple Wishes Hands-Free Breastpump Bra, Pink,...   
44765  Moby Wrap UV SPF 50+ 100% Cotton Baby Carrier,...   
48173  The First Years American Red Cross Deluxe Nail...   

                                                  review  rating  \
80054  I like the idea but the problems:-awkward clos...       2   
44765  This is my 2nd Moby, just wanted another color...       5   
48173  This is the best nail clipper! Definitely reco...       5   

                                            review_clean  sentiment  
80054  I like the idea but the problemsawkward closin...         -1  
44765  This is my 2nd Moby just wanted another color ...          1  
48173  This is the best nail clipper Definitely recom...          1  


In [26]:
sample_test_matrix = vectorizer.transform(sample_test_data['review_clean'])
scores = sentiment_model.decision_function(sample_test_matrix)
print(scores)
print(sentiment_model.predict(sample_test_matrix))

[-0.59780336 16.7241207   5.64588331]
[-1  1  1]


In [33]:
sentiment_model.coefficients

AttributeError: 'LogisticRegression' object has no attribute 'coefficients'